In [2]:
#install dash
pip install jupyter_dash

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.8 MB 3.0 MB/s 
     |████████████████████████████████| 357 kB 37.3 MB/s 
     |████████████████████████████████| 1.6 MB 42.9 MB/s 
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11447 sha256=d41fccee4cac7901841c4b65becd55f4e184d04499edbb87f8391aa887bb279c
  Stored in directory: /root/.cache/pip/wheels/f9/8d/8d/f6af3f7f9eea3553bc2fe6d53e4b287dad18b06a861ac56ddf
Successfully built retrying


In [5]:
#import relevant libraries
import pandas as pd                       #to perform data manipulation and analysis
import numpy as np                        #to cleanse data
from datetime import datetime             #to manipulate dates
import plotly.express as px               #to create interactive charts
import plotly.graph_objects as go         #to create interactive charts
from jupyter_dash import JupyterDash      #to build Dash apps from Jupyter environments
from dash import dcc       #to get components for interactive user interfaces
from dash import html
import math

In [6]:
#read transactional data (created tot test program)
df = pd.read_csv("/content/drive/MyDrive/transaction.csv") 
df.head()

,id,date,amount,category
0,1.0,01-09-2022,8000.0,Salary
1,2.0,01-09-2022,-200.0,Electricity
2,3.0,02-09-2022,-741.4,Restaurant
3,4.0,02-09-2022,-137.0,Mr D Delivery
4,5.0,02-09-2022,-2000.0,Rent


In [10]:
#assign categories to transactions
df['category'] = np.where(df['category'].str.contains('Salary|Extra_income'), 'Income', df['category'] )
df['category'] = np.where(df['category'].str.contains('Mr D Delivery|eat|eats|Checkers|PnP|Groceries|Restaurant'), 'Food', df['category'] )
df['category'] = np.where(df['category'].str.contains('Yoga|Zumba|Exercise'), 'Gym', df['category'] )
df['category'] = np.where(df['category'].str.contains('Shell|Petrol|Bolt'), 'Transport', df['category'] )


In [11]:
#assign date type to date column
df['date'] = pd.to_datetime(df['date'])
df['year_month'] = df['date'].dt.strftime('%Y-%m')

In [12]:
#net worth = income - expenses
Net_Worth_Table = df.groupby('year_month')['amount'].sum().reset_index(name ='sum')
Net_Worth_Table['cumulative sum'] = Net_Worth_Table['sum'].cumsum()
Net_Worth_Chart = go.Figure(
    data = go.Scatter(x = Net_Worth_Table["year_month"], y = Net_Worth_Table["cumulative sum"]),
    layout = go.Layout(
        title = go.layout.Title(text = "Net Worth Over Time")
    )
)
Net_Worth_Chart.update_layout(
    xaxis_title = "Date",
    yaxis_title = "Net Worth (£)",
    hovermode = 'x unified'
    )
Net_Worth_Chart.update_xaxes(
    tickangle = 45)
Net_Worth_Chart.show()

In [18]:
#Calculate expenses
df = df[df.category != "Income"] 
df.amount = df.amount*(-1) 
Total_Monthly_Expenses_Table = df.groupby('year_month')['amount'].sum().reset_index(name = 'sum')
Total_Monthly_Expenses_Chart = px.bar(Total_Monthly_Expenses_Table, x = "year_month", y = "sum", title = "Total Monthly Expenses")
Total_Monthly_Expenses_Chart.update_yaxes(title = 'Expenses (ZAR)', visible = True, showticklabels = True)
Total_Monthly_Expenses_Chart.update_xaxes(title = 'Date', visible = True, showticklabels = True)
Total_Monthly_Expenses_Chart.show()

In [22]:
#Show expense by category
Expenses_Breakdown_Table = pd.pivot_table(df, values = ['amount'], index = ['category', 'year_month'], aggfunc=sum).reset_index()
Expenses_Breakdown_Table.columns = [x.upper() for x in Expenses_Breakdown_Table.columns]
Expenses_Breakdown_Chart = px.line(Expenses_Breakdown_Table, x='YEAR_MONTH', y="AMOUNT", title="Expenses Breakdown", color = 'CATEGORY')
Expenses_Breakdown_Chart.update_yaxes(title='Expenses (ZAR)', visible=True, showticklabels=True)
Expenses_Breakdown_Chart.update_xaxes(title='Date', visible=True, showticklabels=True)
Expenses_Breakdown_Chart.show()

In [24]:
# Build App
app = JupyterDash(__name__)

app.layout = html.Div([   
    html.Div([
        html.H1(" Personal Finance Summary",style={'text-align':'center'}),
        dcc.Graph(figure = Net_Worth_Chart)
    ]),  
    html.Div([
        dcc.Graph(figure = Total_Monthly_Expenses_Chart)
    ]),   
    html.Div([
        dcc.Graph(figure = Expenses_Breakdown_Chart)
    ])
])
    
# Run app and display result
app.run_server(mode='external')


Dash app running on:


<IPython.core.display.Javascript object>